In [ ]:
#|default_exp net.test_graph_mermaid

In [ ]:
#|export
import fbdev

In [ ]:
#|export
from typing import Type, Optional, Union, Any, Tuple, Dict
import asyncio

from fbdev.component import func_component, PortSpecCollection, PortTypeSpec, PortSpec, PortType, DummyComponentFactory
from fbdev.graph import Graph, NodeSpec, EdgeSpec
from fbdev.node import Node, GraphComponentFactory

In [ ]:
#|export
sub_graph = Graph(PortSpecCollection(
    input=PortTypeSpec(subgraph_in=PortSpec(dtype=int)),
    output=PortTypeSpec(subgraph_out=PortSpec(dtype=int))
))
sub_graph_component = GraphComponentFactory.get_component(sub_graph)
sub_graph_node = NodeSpec(sub_graph_component)
dummy_component = DummyComponentFactory.get_component(
    inputs=['in'],
    outputs=['out'],
    component_name='dummy'
)

sub_graph_edge0 = EdgeSpec()
sub_graph.add_edge(sub_graph_edge0)
sub_graph.add_edge(EdgeSpec())
dummy_node = NodeSpec(dummy_component)
sub_graph.add_node(dummy_node)

sub_graph.connect_edge_to_graph_port(PortType.INPUT, 'subgraph_in', 0)
sub_graph.connect_edge_to_node('dummy', PortType.INPUT, 'in', 0)
sub_graph.connect_edge_to_graph_port(PortType.OUTPUT, 'subgraph_out', 1)
sub_graph.connect_edge_to_node('dummy', PortType.OUTPUT, 'out', 1)

top_graph = Graph(PortSpecCollection(
    input=PortTypeSpec(graph_in=PortSpec(dtype=int)),
    output=PortTypeSpec(graph_out=PortSpec(dtype=int))
))

top_graph_edge0 = EdgeSpec()
top_graph.add_edge(top_graph_edge0)
top_graph.add_edge(EdgeSpec())
sub_graph_node = NodeSpec(sub_graph_component)
top_graph.add_node(sub_graph_node, id='subgraph')

top_graph.connect_edge_to_graph_port(PortType.INPUT, 'graph_in', 0)
top_graph.connect_edge_to_node('subgraph', PortType.INPUT, 'subgraph_in', 0)
top_graph.connect_edge_to_graph_port(PortType.OUTPUT, 'graph_out', 1)
top_graph.connect_edge_to_node('subgraph', PortType.OUTPUT, 'subgraph_out', 1)

In [ ]:
sub_graph.display_mermaid()

```mermaid
flowchart 
    classDef subgraph_zone fill:#111
    classDef loose_edge fill:#f00

    subgraph net__CHILD__dummy["dummy[]"]
        net__CHILD__dummy__PORTS__input__CHILD__in{{in}}
        net__CHILD__dummy__PORTS__output__CHILD__out([out])
    end

        net__PORTS__input__CHILD__subgraph_in{{subgraph_in}}
        net__PORTS__output__CHILD__subgraph_out([subgraph_out])

    net__PORTS__input__CHILD__subgraph_in -.-> net__CHILD__dummy__PORTS__input__CHILD__in
    net__CHILD__dummy__PORTS__output__CHILD__out -.-> net__PORTS__output__CHILD__subgraph_out

```

In [ ]:
top_graph.display_mermaid()

```mermaid
flowchart 
    classDef subgraph_zone fill:#111
    classDef loose_edge fill:#f00

    subgraph net__CHILD__subgraph["GraphComponent[subgraph]"]
        subgraph net__CHILD__subgraph__PROP__children[" "]
            subgraph net__CHILD__subgraph__CHILD__dummy["dummy[]"]
                net__CHILD__subgraph__CHILD__dummy__PORTS__input__CHILD__in{{in}}
                net__CHILD__subgraph__CHILD__dummy__PORTS__output__CHILD__out([out])
            end
        end
            net__CHILD__subgraph__PORTS__input__CHILD__subgraph_in{{subgraph_in}}
            net__CHILD__subgraph__PORTS__output__CHILD__subgraph_out([subgraph_out])
    end

        net__PORTS__input__CHILD__graph_in{{graph_in}}
        net__PORTS__output__CHILD__graph_out([graph_out])

    net__CHILD__subgraph__PORTS__input__CHILD__subgraph_in -.-> net__CHILD__subgraph__CHILD__dummy__PORTS__input__CHILD__in
    net__CHILD__subgraph__CHILD__dummy__PORTS__output__CHILD__out -.-> net__CHILD__subgraph__PORTS__output__CHILD__subgraph_out
    net__PORTS__input__CHILD__graph_in -.-> net__CHILD__subgraph__PORTS__input__CHILD__subgraph_in
    net__CHILD__subgraph__PORTS__output__CHILD__subgraph_out -.-> net__PORTS__output__CHILD__graph_out

    class net__CHILD__subgraph__PROP__children subgraph_zone
```

In [ ]:
#|export
def test_address_lookup():
    assert top_graph.get_by_address('>net>subgraph') == sub_graph_node
    assert top_graph.get_by_address('>net>subgraph>dummy') == dummy_node
    assert top_graph.get_by_address('>net>subgraph|0') == sub_graph_edge0
    assert top_graph.get_by_address('>net|0') == top_graph_edge0

In [ ]:
#|export
def test_graph_to_mermaid():
    sub_graph.to_mermaid()
    top_graph.to_mermaid()

In [ ]:
test_address_lookup()
test_graph_to_mermaid()